## Import Libraries

In [1]:
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import os,sys
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint

## Loading Gender Predictor File to predict Gender of LinkedIn Profiles 

In [ ]:
# %load Gender_Prediction.py

# Import Libraries

import os
import re
import urllib2
from zipfile import ZipFile
import csv
import cPickle as pickle

def downloadNames():
    u = urllib2.urlopen('https://www.ssa.gov/oact/babynames/names.zip')
    localFile = open('names.zip', 'w')
    localFile.write(u.read())
    localFile.close()

def getNameList():
    if not os.path.exists('names.pickle'):
        print 'names.pickle does not exist, generating'
        
        # https://www.ssa.gov/oact/babynames/names.zip
        
        if not os.path.exists('names.zip'):
            print 'names.zip does not exist, downloading from github'
            downloadNames()
        else:
            print 'names.zip exists, not downloading'
        
        print 'Extracting names from names.zip'  
        
        namesDict=extractNamesDict()
        
        maleNames=list()
        femaleNames=list()
        
        print 'Sorting Names'
        
        for name in namesDict:
            counts=namesDict[name]
            tuple=(name,counts[0],counts[1])
            if counts[0]>counts[1]:
                maleNames.append(tuple)
            elif counts[1]>counts[0]:
                femaleNames.append(tuple)
        
        names=(maleNames,femaleNames)
        
        print 'Saving names.pickle'
        fw=open('names.pickle','wb')
        pickle.dump(names,fw,-1)
        fw.close()
        print 'Saved names.pickle'
    else:
        print 'names.pickle exists, loading data'
        f=open('names.pickle','rb')
        names=pickle.load(f)
        print 'names.pickle loaded'
        
    print '%d male names loaded, %d female names loaded'%(len(names[0]),len(names[1]))
    
    return names[0],names[1]

def extractNamesDict():
    zf=ZipFile('names.zip', 'r')
    filenames=zf.namelist()
    
    names=dict()
    genderMap={'M':0,'F':1}
    
    for filename in filenames:
        fp = zf.open(filename,'rU')
        rows=csv.reader(fp, delimiter=',', dialect=csv.excel_tab)
        try:
            for row in rows:
            #print name,row[1]
                try:
                    name=row[0].upper()
                    gender=genderMap[row[1]]
                    count=int(row[2])

                    if not names.has_key(name):
                        names[name]=[0,0]

                    names[name][gender]=names[name][gender]+count
                except:
                    pass
        except:
            pass
            
        fp.close()
        
        print '\tImported %s'%filename
    return names


def find_gender_from_first_name(name):
    if name.upper() in new_male_list:
        return "Male"
    elif name.upper() in new_female_list:
        return "Female"
    else:
        return "Unknown"    

if __name__ == '__main__':
    
	male_names, female_names = getNameList()
	new_male_list = []
	new_female_list = []

	for index,name in enumerate(male_names):
		try:
		    if (name[1]/name[2])>=4:
		        new_male_list.append(name[0])
		except:
		    new_male_list.append(name[0])
		    
	#print "Total number of Male Names after is %d." %len(new_male_list)	
	
	
	for index,name in enumerate(female_names):
		try:
		    if (name[2]/name[1])>=4:
		        new_female_list.append(name[0])
		except:
		    new_female_list.append(name[0])
		    
	#print "Total number of Female Names after is %d." %len(new_female_list)
	
	#find_gender_from_first_name('Harsh')
	


## Get Link of Profile Picture

In [4]:
def getProfilePicLink(html):
       
    soup=BeautifulSoup(html,"lxml") 
    images = [x for x in soup.find_all('img')]
    #print images
    try:
        if "shrinknp_200_200" in str(images[0]):
            imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
        else:
            imageUrlString = ""
    except:
        imageUrlString = ""
    
    #print imageUrlString
    
    return imageUrlString

## Store Profile Picture to Local Directory

In [5]:
def storeProfilePicture(profileUrl,profile_link):
    
    lst = profileUrl.split()
    userId = profile_link.split('/')[-1]
    regex=re.compile(r'(src).*')
    img_url = re.sub('src=','', "".join([m.group(0) for l in lst for m in [regex.search(l)] if m]))
    img_url = img_url.strip('""')
    #print img_url
    if img_url:
        urllib.urlretrieve(img_url, "Images/" + userId + ".jpg")
        print userId + ".jpg is saved."
        return img_url
    else:
        with open('ghost_person.png', 'rb') as f:
            data = f.read()

        with open("Images/" + userId + ".png", 'wb') as f:
            f.write(data)

        print userId + ".png is saved."
        return "https://static.licdn.com/scds/common/u/images/themes/katy/ghosts/person/ghost_person_100x100_v1.png".strip("")

## Scrap the UserID from Recommended Users' list. 

In [6]:
def getRecommendedUserIds(html):
    
    
    soup=BeautifulSoup(html,"lxml")
    #content = driver.page_source
    #images = [x for x in soup.find_all('img')]
    profLinks = [x for x in soup.find_all('li',{'class': 'profile-card'})]
    
    recUserIds = []
    #print profLinks
    for link in profLinks:
        urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(link))
        recId = urls[0].split('?')[0].split('/')[-1]
        recUserIds.append(recId)
    
    return recUserIds

## Get Full Name from title in source

In [7]:
def getName(html):
    
    soup=BeautifulSoup(html,"lxml")
    title = soup.find('title')
    name = str(title).replace('<title>','')
    full_name = name.replace(' | LinkedIn</title>','')
    
    return full_name

## Get All Bachelor Degree List and Make a dictionary

In [8]:
def getBachelorList():

    BachelorDict = {}
    regex = re.compile('[^a-zA-Z/]')
    with open('bachelor_degrees.txt') as fp:
        for line in fp.readlines():
            lineSepator = line.split('(')
            abbr = regex.sub('', lineSepator[1])
            abbr = abbr.split('/')
            for abrv in abbr:
                BachelorDict[abrv] = lineSepator[0].strip()
        fp.close()

    return BachelorDict

## Calculate Age from Bachelor Degree Starting Year

In [9]:
def calculate_age(bachelor_year):
    today = date.today()
    return today.year - bachelor_year + 18

## Find Person's All Degrees and their Duration 

In [10]:
def get_Degree_Duration(html):
    
    soup=BeautifulSoup(html.encode("ascii","ignore"),"lxml")
    schoolLinks = soup.find_all('li',{'class':'school'})
    degreeList = []
    time_range_list = []
    
    for soup1 in schoolLinks:

        degreeLink = soup1.find('span',{'data-field-name':"Education.DegreeName"})
        timeRange = soup1.find('span',{'class':"date-range"})

        tempDegree = str(degreeLink).replace('<span class="translated translation" data-field-name="Education.DegreeName">','')
        degree = tempDegree.replace('</span>','')

        tempTime = str(timeRange).replace('<span class="date-range">','')
        time = tempTime.replace('<time>','')
        temp_time = time.replace('</time>','')
        time_range = temp_time.replace('</span>','')

        degreeList.append(degree)
        time_range_list.append(time_range)
    
    return degreeList,time_range_list

## Find the Bachelor Year from Degree List and its Duration

In [11]:
def find_bachelor_year(degree_list,time_list):
    
    bachelor_degree_duration = set()
    
    BachelorDict = getBachelorList()
    
    for index,dg in enumerate(degree_list):

        for key in BachelorDict.keys():
            if key in dg:
                bachelor_degree_duration.add(time_list[index])
                break

        for value in BachelorDict.values():
            if value in dg:
                bachelor_degree_duration.add(time_list[index])
                break
        
        
    #print time_list   
    bachelor_degree_duration = list(bachelor_degree_duration)
    
    #print bachelor_degree_duration[0]
    
    try:
        if bachelor_degree_duration[0] == 'None':
            if time_list:
                #print time_list
                bachelor_year = int(time_list[0].split()[0]) - 5
            else:
                bachelor_year = None

        else:
            bachelor_year = int(bachelor_degree_duration[0].split()[0])    
    
    except:
        bachelor_year = None
    

    return bachelor_year

## Find the age from LinkedIn Profile

In [12]:
def age_from_linkedin_profile(profileUrl):
    
    try:
        degree_list, time_list = get_Degree_Duration(profileUrl)
    
        refined_degree_list = []
        regex = re.compile('[^a-zA-Z\s+]')

        for degree in degree_list:
            refined_degree_list.append(regex.sub('',degree))

        bachelor_year = find_bachelor_year(refined_degree_list,time_list)
        #print bachelor_year

        if bachelor_year:
            age = calculate_age(bachelor_year)
        else:
            age = None
    
    except:
        age = None
        
    return age

In [41]:
pkl_file = open("linkedin_profiles.pickle","rb")
myList=pickle.load(pkl_file) # errors out here
pkl_file.close()

In [14]:
myList

[{'harshparikh1001': {'Full_Name': 'Harsh Parikh',
   'Gender': 'Male',
   'Profile_Url': 'https://media.licdn.com/mpr/mpr/shrinknp_400_400/AAEAAQAAAAAAAAmGAAAAJDNhZmE5ODljLTZjMDItNGUyNC1iYjYyLWMxYTJiNzE1NmYzNA.jpg',
   'Recommended_Ids': ['harshparikh',
    'parikhharsh',
    'harsh-parikh-83976879',
    'parikhharsh86',
    'harshparikhhp',
    'singhlatika',
    'vivek-basavarajappa-81a270b1',
    'dktrivedi265',
    'shouviksdutta',
    'sanketwagh',
    'parthshah7',
    'karthikbangalore',
    'dpatel0220',
    'ramyamadhihallysreenidhi',
    'xulifeng'],
   'age': 24}}]

In [21]:
userIds = myList[0]['harshparikh1001']['Recommended_Ids']
userIds

['harshparikh',
 'parikhharsh',
 'harsh-parikh-83976879',
 'parikhharsh86',
 'harshparikhhp',
 'singhlatika',
 'vivek-basavarajappa-81a270b1',
 'dktrivedi265',
 'shouviksdutta',
 'sanketwagh',
 'parthshah7',
 'karthikbangalore',
 'dpatel0220',
 'ramyamadhihallysreenidhi',
 'xulifeng']

## Main Method

In [22]:
if __name__ == '__main__':
    
    directory = "Images"
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    recommended_profile_ids = []
    
    
    profiles = []
    
    output = open("linkedin_profiles1.pickle", 'wb')   # Save all profiles as pickle file
    
    for index,usrid in enumerate(userIds):
        
        profileUrl = "https://www.linkedin.com/in/" + usrid
        
        driver = webdriver.PhantomJS(service_log_path=os.path.devnull)
        driver.get(profileUrl)
        html=driver.page_source
        
        profileDict = {}
        
        profileDict['User_ID'] = usrid
        profileDict['Full_Name'] = getName(html)
        profileDict['Gender'] = find_gender_from_first_name(getName(html).split()[0])
        profileDict['Recommended_Ids'] = getRecommendedUserIds(html)
        
        profilePicUrl = getProfilePicLink(html)
        
        picUrl = storeProfilePicture(profilePicUrl,profileUrl)
        
        profileDict['Profile_Url'] = picUrl
        profileDict['age'] = age_from_linkedin_profile(html)
        
        profiles.append(profileDict)
        
        
        if index % 5 == 0:
            time.sleep(20) # delays for 5 seconds
        
    pickle.dump(profiles, output,-1)
    output.close()

harshparikh.jpg is saved.
parikhharsh.png is saved.
harsh-parikh-83976879.jpg is saved.
parikhharsh86.jpg is saved.
harshparikhhp.jpg is saved.
singhlatika.jpg is saved.
vivek-basavarajappa-81a270b1.png is saved.
dktrivedi265.jpg is saved.
shouviksdutta.jpg is saved.
sanketwagh.jpg is saved.
parthshah7.jpg is saved.
karthikbangalore.jpg is saved.
dpatel0220.jpg is saved.
ramyamadhihallysreenidhi.jpg is saved.
xulifeng.jpg is saved.


## Open pickle file and read stuff from it

In [39]:
pkl_file = open("linkedin_profiles1.pickle","rb")
my_new_list=pickle.load(pkl_file) # errors out here
pkl_file.close()

In [45]:
len(my_new_list)

15

In [47]:
profile_list = my_new_list + myList

In [48]:
len(profile_list)

19

In [36]:
pkl_file = open("linkedin_profiles.pickle","wb")
pickle.dump(profile_list, pkl_file,-1) # errors out here
pkl_file.close()

In [56]:
for index,profile in enumerate(profile_list):
    if profile['User_ID'] == 'Harsh Parikh':
        print profile

{'User_ID': 'harshparikh', 'Gender': 'Male', 'age': 39, 'Recommended_Ids': ['parikhharsh', 'harsh-parikh-83976879', 'parikhharsh86', 'harshparikhhp', 'harshparikh1001', 'stephen-kolano-cfa-5a829425', 'steven-dupr%C3%A9-859110', 'michael-schlachter-cfa-a68b2467', 'karen-mcquiston-45872961', 'christopher-krug-cfa-8231488', 'keshav-rajagopalan-a1a66165', 'cleo-chang-0b4b6823', 'timothy-lalevee-53421216', 'tully-cheng-fsa-cfa-802b5522'], 'Full_Name': 'Harsh Parikh', 'Profile_Url': 'https://media.licdn.com/mpr/mpr/shrinknp_400_400/AAEAAQAAAAAAAAd1AAAAJDE2NGVlMTM4LTk2NjYtNDk0Ni1iZGYzLWMyYWU4Y2MwZGY1YQ.jpg'}
{'User_ID': 'parikhharsh', 'Gender': 'Male', 'age': None, 'Recommended_Ids': ['harshparikh', 'harsh-parikh-83976879', 'parikhharsh86', 'harshparikhhp', 'harshparikh1001', 'richardholm1', 'colinmproksel', 'laura-schwartz-257674b', 'ian-peck-0940632a', 'jordan-ludwig-384b453a', 'shuchee-shah-8463128a', 'april-ham-87095a54', 'brent-westcott-5ab9003b', 'christina-goodrich-5576b758', 'joellesi

In [66]:
uids = [uid for uid in (d['User_ID'] for d in profile_list)]   

In [67]:
uids

['harshparikh',
 'parikhharsh',
 'harsh-parikh-83976879',
 'parikhharsh86',
 'harshparikhhp',
 'singhlatika',
 'vivek-basavarajappa-81a270b1',
 'dktrivedi265',
 'shouviksdutta',
 'sanketwagh',
 'parthshah7',
 'karthikbangalore',
 'dpatel0220',
 'ramyamadhihallysreenidhi',
 'xulifeng',
 'harshparikh1001',
 'marissamayer',
 'williamhgates',
 'satya-nadella-3145136']

## Test profile of "Barbara Corcoran"

https://www.linkedin.com/in/barbaracorcoran

In [84]:
driver = webdriver.PhantomJS(service_log_path=os.path.devnull)
driver.get("https://www.linkedin.com/in/barbaracorcoran")
html3=driver.page_source  
#print html
soup=BeautifulSoup(html3,"lxml") 
images = [x for x in soup.find_all('img')]
#print images

In [78]:
#imageUrlString

In [85]:
find_gender_from_first_name(getName(html3).split()[0])

'Female'

In [86]:
print age_from_linkedin_profile(html3)

None


## My Profile and Age

https://www.linkedin.com/in/harshparikh1001

In [77]:
driver = webdriver.PhantomJS(service_log_path=os.path.devnull)
driver.get("https://www.linkedin.com/in/harshparikh1001")
html1=driver.page_source  
#print html1
soup=BeautifulSoup(html1,"lxml") 
images = [x for x in soup.find_all('img')]
#print images
try:
    if "shrinknp_200_200" in str(images[0]):
        imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
    else:
        imageUrlString = ""
except:
    imageUrlString = ""

In [79]:
find_gender_from_first_name(getName(html1).split()[0])

'Male'

In [80]:
age_from_linkedin_profile(html1)

24

## Test profile of "Satya Nadella"

https://www.linkedin.com/in/satya-nadella-3145136

In [81]:
driver = webdriver.PhantomJS(service_log_path=os.path.devnull)
driver.get("https://www.linkedin.com/in/satya-nadella-3145136")
html2=driver.page_source  
#print html2
soup=BeautifulSoup(html2,"lxml") 
images = [x for x in soup.find_all('img')]
#print images
try:
    if "shrinknp_200_200" in str(images[0]):
        imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
    else:
        imageUrlString = ""
except:
    imageUrlString = ""

In [82]:
find_gender_from_first_name(getName(html2).split()[0])

'Unknown'

In [83]:
age_from_linkedin_profile(html2)

45

In [78]:
driver = webdriver.PhantomJS(service_log_path=os.path.devnull)
driver.get("https://www.linkedin.com/in/karthikbangalore")
html1=driver.page_source  
#print html1
soup=BeautifulSoup(html1,"lxml") 
images = [x for x in soup.find_all('img')]
#print images
try:
    if "shrinknp_200_200" in str(images[0]):
        imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
    else:
        imageUrlString = ""
except:
    imageUrlString = ""

In [79]:
imageUrlString

'<img alt="Karthik Bangalore Mani" class="image photo lazy-loaded" height="200" lazyload="true" src="https://media.licdn.com/mpr/mpr/shrinknp_400_400/AAEAAQAAAAAAAASlAAAAJDVmZDNjYzA1LWNlYzQtNDk4ZS1hMTEyLTcxMGRhNDczMjFlOA.jpg" width="200"/>'

In [ ]:


#driver = webdriver.PhantomJS()
#driver.get("https://www.linkedin.com/in/georgecorliss")
#html=driver.page_source    
#soup=BeautifulSoup(html,"lxml")
#schoolLinks = soup.find_all('li',{'class':'school'})

#for soup1 in schoolLinks:
    
#    degreeLink = soup1.find('span',{'data-field-name':"Education.DegreeName"})
#    timeRange = soup1.find('span',{'class':"date-range"})
    
#    tempDegree = str(degreeLink).replace('<span class="translated translation" data-field-name="Education.DegreeName">','')
#    degree = tempDegree.replace('</span>','')
    
#    tempTime = str(timeRange).replace('<span class="date-range">','')
#    time = tempTime.replace('<time>','')
#    temp_time = time.replace('</time>','')
#    time_range = temp_time.replace('</span>','')
    
#    print degree
#    print time_range

In [ ]:
#import re
#link = re.sub(r'(<a href="http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+">)','',str(links[3]))
#link = link.replace('<ul>',' ')
#link = link.replace('</li><li>',' ')
#link = link.replace('<li>',' ')
#link = link.replace('</a>',' ')
#link = link.replace('</li></ul>',' ')
#print link.split()

## Link to get all profiles with the first Name

https://www.linkedin.com/pub/dir/Bill